### Comparison between Automatic Prompt Engineering (APE) and ReAct (Reasoning + Acting)

In [1]:
from _pipeline import create_payload, model_req
import time

### Step 1: Define PETs
PET_1 = "Automatic Prompt Engineering (APE)"
PET_2 = "ReAct (Reasoning + Acting)"

#### Step 2: Define Use Case (Requirement Analysis for a Chatbot)
USER_REQUEST = """
Generate a requirement analysis for a Discord chatbot that:
- Enables data scientists and business analysts to efficiently locate and extract relevant data from company databases to answer specific queries and generate reports.
"""

#### Step 3: Level-1 Automation — Generate Optimized Prompts with APE
ape_meta_prompt = f"""
You are an expert in Automatic Prompt Engineering (APE). Generate an optimized prompt that can be used to extract a detailed requirement analysis for a Discord chatbot.

User Request:
{USER_REQUEST}

Make sure the generated prompt is structured, detailed, and effective.
"""

ape_payload = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=ape_meta_prompt,
    temperature=1.0,
    num_ctx=300,
    num_predict=400,
)

ape_time, ape_generated_prompt = model_req(payload=ape_payload)
print("\n### Generated Prompt Using APE ###\n", ape_generated_prompt)
if ape_time:
    print(f"APE Processing Time: {ape_time}s")

#### Step 4: Level-1 Automation — Generate ReAct Prompt
react_meta_prompt = f"""
You are an expert in Reasoning + Acting (ReAct). Given the following user request, generate a detailed requirement analysis in an iterative step-by-step format.

User Request:
{USER_REQUEST}

Your response should include:
1. Thought Process: How should the chatbot be designed?
2. Action: Generate the requirement analysis.
3. Reflection: Identify gaps and refine the requirement iteratively.
"""

react_payload = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=react_meta_prompt,
    temperature=1.0,
    num_ctx=300,
    num_predict=400,
)

react_time, react_generated_prompt = model_req(payload=react_payload)
print("\n### Generated Prompt Using ReAct ###\n", react_generated_prompt)
if react_time:
    print(f"ReAct Processing Time: {react_time}s")

#### Step 5: Level-2 Automation — Generate Requirement Analysis for Both PETs
parameters = [
    {"temperature": 0.7, "num_ctx": 500, "num_predict": 700},  # More deterministic
    {"temperature": 1.2, "num_ctx": 600, "num_predict": 800},  # More creative
]

responses = {}

for i, prompt in enumerate([ape_generated_prompt, react_generated_prompt]):
    for j, param_set in enumerate(parameters):
        config = f"PET-{i+1}, Config-{j+1}"
        print(f"\n### Running {config} ###")

        payload = create_payload(
            target="ollama",
            model="llama3.2:latest",
            prompt=prompt,
            temperature=param_set["temperature"],
            num_ctx=param_set["num_ctx"],
            num_predict=param_set["num_predict"],
        )

        start_time = time.time()
        _, response = model_req(payload=payload)
        execution_time = round(time.time() - start_time, 2)

        print(response)

        responses[config] = {
            "Response": response,
            "Execution Time": execution_time,
            "Params": param_set,
        }

{'model': 'llama3.2:latest', 'prompt': '\nYou are an expert in Automatic Prompt Engineering (APE). Generate an optimized prompt that can be used to extract a detailed requirement analysis for a Discord chatbot.\n\nUser Request:\n\nGenerate a requirement analysis for a Discord chatbot that:\n- Enables data scientists and business analysts to efficiently locate and extract relevant data from company databases to answer specific queries and generate reports.\n\n\nMake sure the generated prompt is structured, detailed, and effective.\n', 'stream': False, 'options': {'temperature': 1.0, 'num_ctx': 300, 'num_predict': 400}}

### Generated Prompt Using APE ###
 Here's an optimized prompt to extract a detailed requirement analysis for a Discord chatbot:

**Prompt:**

"Design a comprehensive requirement analysis document for a Discord chatbot that enables data scientists and business analysts to efficiently locate and extract relevant data from company databases. The chatbot should allow users 

In [2]:
#### Step 6: Comparative Analysis & Performance Evaluation
analysis_report = "### Comparative Analysis of Prompt Engineering Techniques ###\n\n"

for config, details in responses.items():
    analysis_report += f"\n#### {config} ####\n"
    analysis_report += f"**Execution Time:** {details['Execution Time']}s\n"
    analysis_report += f"**Temperature:** {details['Params']['temperature']}\n"
    analysis_report += f"**Response Length:** {len(details['Response'])} characters\n"
    analysis_report += f"**Response Preview:** {details['Response'][:300]}...\n"

# Compare results based on execution time and response length
best_config = min(responses, key=lambda x: responses[x]["Execution Time"])

analysis_report += f"\n\n### Best Performing Configuration: {best_config} ###\n"
analysis_report += (
    f"**Shortest Execution Time:** {responses[best_config]['Execution Time']}s\n"
)
analysis_report += (
    f"**Response Length:** {len(responses[best_config]['Response'])} characters\n"
)
analysis_report += f"**Parameters:** {responses[best_config]['Params']}\n"

print("\n### Final Comparative Analysis Report ###\n", analysis_report)


### Final Comparative Analysis Report ###
 ### Comparative Analysis of Prompt Engineering Techniques ###


#### PET-1, Config-1 ####
**Execution Time:** 15.83s
**Temperature:** 0.7
**Response Length:** 3677 characters
**Response Preview:** Based on the provided prompt, I'll outline a detailed requirement analysis document for the Discord chatbot.

**Requirement Analysis Document**

**Introduction:**
The goal of this chatbot is to provide an efficient and user-friendly interface for data scientists and business analysts to extract rele...

#### PET-1, Config-2 ####
**Execution Time:** 15.02s
**Temperature:** 1.2
**Response Length:** 3551 characters
**Response Preview:** Based on the provided prompt, here's a comprehensive requirement analysis document for a Discord chatbot:

**Requirements Analysis Document**

**Overview**
----------------

The proposed Discord chatbot aims to empower data scientists and business analysts to efficiently locate and extract relevant ...

#### PET-2, Conf